# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [12]:
import os
import glob
import psycopg2
import pandas as pd
import numpy as np
from sql_queries import *

In [13]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [14]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [15]:
# Get All File Names
song_files =get_files("data/song_data/")

In [16]:
# Print all the files
for song in range(len(song_files)):
    print(song_files[song])

/home/workspace/data/song_data/A/A/A/TRAAAVG12903CFA543.json
/home/workspace/data/song_data/A/A/A/TRAAADZ128F9348C2E.json
/home/workspace/data/song_data/A/A/A/TRAAAVO128F93133D4.json
/home/workspace/data/song_data/A/A/A/TRAAARJ128F9320760.json
/home/workspace/data/song_data/A/A/A/TRAAAAW128F429D538.json
/home/workspace/data/song_data/A/A/A/TRAAABD128F429CF47.json
/home/workspace/data/song_data/A/A/A/TRAAAMO128F1481E7F.json
/home/workspace/data/song_data/A/A/A/TRAAAMQ128F1460CD3.json
/home/workspace/data/song_data/A/A/A/TRAAAPK128E0786D96.json
/home/workspace/data/song_data/A/A/A/TRAAAEF128F4273421.json
/home/workspace/data/song_data/A/A/A/TRAAAFD128F92F423A.json
/home/workspace/data/song_data/A/A/A/.ipynb_checkpoints/TRAAAAW128F429D538-checkpoint.json
/home/workspace/data/song_data/A/A/A/.ipynb_checkpoints/TRAAABD128F429CF47-checkpoint.json
/home/workspace/data/song_data/A/A/A/.ipynb_checkpoints/TRAAAPK128E0786D96-checkpoint.json
/home/workspace/data/song_data/A/A/A/.ipynb_checkpoints/

In [17]:
#Select the first file
filepath = song_files[0]

In [18]:
type(song_files)

list

In [28]:
df=[]
for f in song_files:
    df=pd.read_json(f,lines=True)

In [29]:
print(df)

            artist_id  artist_latitude artist_location  artist_longitude  \
0  ARAGB2O1187FB3A161              NaN                               NaN   

                       artist_name   duration  num_songs             song_id  \
0  Pucho & His Latin Soul Brothers  338.23302          1  SOLEYHO12AB0188A85   

                title  year  
0  Got My Mojo Workin     0  


In [27]:
tdf = pd.DataFrame([df], columns=['song_id', 'title','artist_id', 'year', 'duration', 'artist_name', 'artist_location', 'artist_latitude','artist_longitude'])
tdf.head()

ValueError: Shape of passed values is (1, 1), indices imply (9, 1)

In [ ]:
#Print the fisrt file
print(filepath)

In [ ]:
# Read the JSON file line by line when lines=True and See the dataframe first 5 records using head method
df = pd.read_json(filepath ,lines=True)
df.head()

## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [30]:
# Print the first record of data frame
song_data = df.values[0]
print(song_data)

['ARAGB2O1187FB3A161' nan '' nan 'Pucho & His Latin Soul Brothers'
 338.23302 1 'SOLEYHO12AB0188A85' 'Got My Mojo Workin' 0]


In [ ]:
# check the data type of song_data
type(song_data)

In [ ]:
#convert dataframe to list by selecting first record using iloc[0] which is index based, I could use Label based that is loc['song_id']
song_data =list(df[['song_id', 'title','artist_id', 'year', 'duration']].iloc[0])
song_data

In [ ]:
# check the data type of song_data
type(song_data)

In [ ]:
# was getting can't adapt type 'numpy.int64. Not able to convert into Python Type. That's why I convrted into float type
song_data[3]=song_data[3].astype(float)

In [ ]:
# check the data type of song_data list
for n in range(len(song_data)):
    print(type(song_data[n]))

In [ ]:
# verified all the cloumns
cols = df.columns
cols

In [ ]:
# verified all the indexes
ind=df.index
ind

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [ ]:
# below code snippet is passing SQL Insert Query and Song_data as values to execute method. once it is executed and it will write into song dimension table and do commit.
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [ ]:
#convert dataframe to list by selecting first record using iloc[0] which is index based.
artist_data = list(df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude','artist_longitude']].iloc[0])
artist_data

In [ ]:
# checking the data types of list
for n in range(len(artist_data)):
    print(type(artist_data[n]))

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [ ]:
# below code snippet is passing SQL Insert Query and artist_data as values to execute method. once it is executed and it will write into artist dimension table and do commit.
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [ ]:
log_files = get_files("data/log_data")

In [ ]:
filepath = log_files[0]

In [ ]:
print(filepath)

In [ ]:
df = pd.read_json(filepath,lines=True)
df.head()

In [ ]:
df.columns

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [ ]:
#Filter the records by Nextsong
df = df[df.page=='NextSong']
#df.head()

In [ ]:
#convert the dataframe ts column to datetime
df['ts']=pd.to_datetime(df['ts'])

In [ ]:
#Created another DataFrame and assigned to temporary dataframe (t)
t=pd.DataFrame()
#assigned the df[ts] to t[ts]
t['ts']=df['ts']

#Extract the timestamp, hour, day, week of year, month, year, and weekday from the ts column and set to t dataframe
t['timestamp'] = t['ts'].dt.time
t['hour'] = t['ts'].dt.hour
t['day'] = t['ts'].dt.day
t['week of year'] = t['ts'].dt.week
t['month'] = t['ts'].dt.month
t['year'] = t['ts'].dt.year
t['weekday'] = t['ts'].dt.dayofweek 
t.set_index('ts',inplace=True)
#t.drop(['ts'], axis=1, inplace=True)
t.head()

In [ ]:
type(t)

In [ ]:
#convert t dataframe to list
time_data = t.values.tolist() #[df.columns.values.tolist()] 
#label the list with column names
column_labels = ('timestamp','hour','day','week of year','month','year','weekday')

In [ ]:
#convert the list into dataframe by providing the list and column names
time_df = pd.DataFrame(time_data,columns=column_labels)
time_df.head()

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [ ]:
#loop through the each row of datafream and insert into time dimension table
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [ ]:
user_df = pd.DataFrame(df,columns=['userId','firstName','lastName','gender','level'])

In [ ]:
user_df.head()

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [ ]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [ ]:
#to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None 
    
    conn.commit()

In [ ]:
df.columns

In [ ]:
# Assign the songid and artistid to dataframe
df['songid']=songid
df['artistid']=artistid
df.head()   

In [ ]:
type(results)

In [ ]:
#insert into songplays table
songplay_data = pd.DataFrame(df,columns=['ts','userId','level','songid','artistid','sessionId','location','userAgent','song','artist','length'])
for i, row in songplay_data.iterrows():
    cur.execute(songplay_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [ ]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.